# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1
print('question 1' + '\n' )
# Example usage:
limit = 5
generator = square_root_generator(limit)
sum=0

for sqrt_value in generator:
    sum=sum+sqrt_value
    print(sum)

print('\n' * 1)
print('question 2' + '\n' )
# Example usage:
limit = 13
generator = square_root_generator(limit)


for sqrt_value in generator:

    print(sqrt_value)


question 1

1.0
2.414213562373095
4.146264369941973
6.146264369941973
8.382332347441762


question 2

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [29]:
person1=people_1()
person3=people_1()
person5=people_1()
person7=people_1()


In [30]:
person2=people_2()
person4=people_2()
person6=people_2()
person8=people_2()

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [5]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX

In [26]:
pipeline1 = dlt.pipeline(destination='duckdb', dataset_name='personal')

# run the pipeline with default settings, and capture the outcome
info = pipeline1.run(person5,
										table_name="person_append2",
										write_disposition="append",
                    primary_key='ID')


info = pipeline1.run(person8,
										table_name="person_append2",
										write_disposition="append",
                    primary_key='ID')



conn = duckdb.connect(f"{pipeline1.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline1.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))



print("\n\n\n person_append table below")
persons = conn.sql("SELECT * FROM person_append2").df()
display(persons)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ person_append       │
│ person_append1      │
│ person_append2      │
│ person_merged       │
└─────────────────────┘




 person_append table below


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708251037.2582042,o+uIEgb1mhdvkQ,None
1,2,Person_2,27,City_A,1708251037.2582042,i751QF5uB5xn5w,None
2,3,Person_3,28,City_A,1708251037.2582042,lRftQWso5xxlWw,None
3,4,Person_4,29,City_A,1708251037.2582042,6eqoJVKLJ9v+5A,None
4,5,Person_5,30,City_A,1708251037.2582042,JTiw0qnr/PKCuQ,None
5,3,Person_3,33,City_B,1708251037.9713628,ZVjOOoEISXjIbA,Job_3
6,4,Person_4,34,City_B,1708251037.9713628,UHo95a0Xr1Vk/A,Job_4
7,5,Person_5,35,City_B,1708251037.9713628,pKAz7fxElQ2L9Q,Job_5
8,6,Person_6,36,City_B,1708251037.9713628,CF8nH8E8HKBuyg,Job_6
9,7,Person_7,37,City_B,1708251037.9713628,AAP+Awto0z+NTw,Job_7


In [31]:
pipeline3 = dlt.pipeline(destination='duckdb', dataset_name='personal')

# run the pipeline with default settings, and capture the outcome
info = pipeline3.run(person5,
										table_name="person_merged1",
										write_disposition="merge",
                    primary_key='ID')


info = pipeline3.run(person8,
										table_name="person_merged1",
										write_disposition="merge",
                    primary_key='ID')




conn = duckdb.connect(f"{pipeline3.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline3.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))



print("\n\n\n person_append table below")
persons = conn.sql("SELECT * FROM person_merged1").df()
display(persons)



Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ person_append       │
│ person_append1      │
│ person_append2      │
│ person_merged       │
│ person_merged1      │
└─────────────────────┘




 person_append table below


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708251135.3936276,kmOG2laEDnHQdA,None
1,2,Person_2,27,City_A,1708251135.3936276,skkV+4dXepXoGw,None
2,8,Person_8,38,City_B,1708251136.311845,2Z5tcVNCFRGcqw,Job_8
3,7,Person_7,37,City_B,1708251136.311845,w5RReWL3Lc29+w,Job_7
4,3,Person_3,33,City_B,1708251136.311845,KG0R5vz0A1UTsw,Job_3
5,6,Person_6,36,City_B,1708251136.311845,AqDHhb0Pge+1bg,Job_6
6,5,Person_5,35,City_B,1708251136.311845,Zh+lovl1bBYo0w,Job_5
7,4,Person_4,34,City_B,1708251136.311845,xJ2YZmif5jggSQ,Job_4


In [34]:
summed = conn.sql("""
SELECT SUM(age) as sum_age_appended
FROM person_append2
""").df()
display(summed)


summed = conn.sql("""
SELECT SUM(age) as sum_age_merged
FROM person_merged1
""").df()
display(summed)

,sum_age_appended
0,353.0


,sum_age_merged
0,266.0
